In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
from gpn.data.dataset_manager import DatasetManager
from gpn.data.dataset_provider import InMemoryDatasetProvider


dataset = "AmazonPhotos"

def load_dataset(dataset):
    dataset_provider = InMemoryDatasetProvider(
        DatasetManager(
            dataset=dataset,
            split_no=1,
            root="./data",
            ood_flag=False,
            train_samples_per_class=0.05,
            val_samples_per_class=0.15,
            test_samples_per_class=0.8,
            split="random",
            # ood_setting="poisoning",
            # ood_type="leave_out_classes",
            # ood_num_left_out_classes=-1,
            # ood_leave_out_last_classes=True,
        )
    )
    
    return dataset_provider

data = load_dataset(dataset).data_list[0]

In [74]:

import train_and_eval as tae

def get_config_name(model):
    if model in ("gpn", "gpn_rw", "gpn_lop"):
        return "configs/gpn/classification_gpn_16.yaml"
    return f"configs/reference/classification_{model}.yaml"

def get_config_updates(model, dataset):
    updates = {}
    match model:
        case "gpn":
            model_name = "GPN"
        case "gpn_rw":
            model_name = "GPN"
            updates["model.adj_normalization"] = "rw"
        case "gpn_lop":
            model_name = "GPN_LOP"
            updates["model.sparse_x_prune_threshold"] = 0.01
        case _:
            model_name = model.upper()
    
    if dataset in ("AmazonPhotos", "AmazonComputers", "PubMedFull"):
        updates["model.sparse_propagation"] = True        
    
    return {
        "model.model_name": model_name,
        "data.dataset": dataset,
        "run.num_inits": 1,
        "run.num_splits": 1,
        "run.log": False,
        "run.job": "predict",
        **updates
    }

def get_prediction(model, dataset):
    res = tae.ex.run(
        named_configs=[get_config_name(model)],
        config_updates={
            **get_config_updates(model, dataset),
            # "run.reduced_training_metrics": True,
            # "training.eval_every": 10,
            # "training.stopping_patience": 5,
            # "data.split": "public",
            # "run.num_splits": 1,
            # "model.model_name": "GPN_LOP",
            # "model.sparse_x_prune_threshold": 0.01,
            # "run.reeval": True,
        },
        options={"--force": True},
    )

    assert isinstance(res.result, list) and len(res.result) == 1
    return res.result[0]

# gpn_rw_pred = get_prediction("gpn_rw", dataset)
gpn_lop_pred = get_prediction("gpn_lop", dataset)

WARNING - train_and_eval - No observers have been added to this run
INFO - train_and_eval - Running command 'run_experiment'
INFO - train_and_eval - Started
INFO - root - Received the following configuration:
INFO - root - RUN
INFO - root - {'experiment_name': 'classification', 'experiment_directory': './saved_experiments', 'reduced_training_metrics': False, 'eval_mode': 'default', 'job': 'predict', 'save_model': True, 'gpu': 0, 'num_inits': 1, 'num_splits': 1, 'log': False, 'debug': True, 'ex_type': 'transductive', 'ood_loc': True, 'ood_loc_only': False, 'ood_edge_perturbations': True, 'ood_isolated_perturbations': False}
INFO - root - -----------------------------------------
INFO - root - DATA
INFO - root - {'to_sparse': False, 'split_no': 1, 'dataset': 'AmazonPhotos', 'root': './data', 'split': 'random', 'train_samples_per_class': 0.05, 'val_samples_per_class': 0.15, 'test_samples_per_class': 0.8, 'ood_flag': False}
INFO - root - -----------------------------------------
INFO - roo

Starting experiment (model=GPN, dataset=AmazonPhotos, ood_type=None, split=1, init=1, results=./saved_experiments/classification/GPN/74/results_1.json, trained=True, evaluated=False).
Completed experiment (model=GPN, dataset=AmazonPhotos, ood_type=None, split=1, init=1, results=./saved_experiments/classification/GPN/74/results_1.json, trained=True, evaluated=False).


INFO - train_and_eval - Result: [Prediction(soft=tensor([[6.9633e-03, 6.7221e-02, 9.2044e-03,  ..., 5.8085e-03, 4.4388e-01,
         1.6319e-01],
        [2.6065e-03, 4.6030e-02, 8.2073e-03,  ..., 2.6013e-03, 1.4444e-01,
         6.7511e-02],
        [5.5774e-04, 4.2731e-02, 2.2177e-03,  ..., 1.5024e-04, 8.6556e-03,
         2.3246e-02],
        ...,
        [7.1430e-03, 9.8327e-01, 8.1609e-04,  ..., 5.1573e-04, 8.1070e-04,
         1.4107e-03],
        [7.6392e-04, 9.5423e-03, 9.3789e-01,  ..., 1.2979e-03, 5.3490e-03,
         2.7091e-02],
        [3.9126e-03, 8.8131e-02, 7.4786e-03,  ..., 1.4017e-03, 1.6701e-01,
         9.8231e-02]]), log_soft=tensor([[-4.9671, -2.6998, -4.6881,  ..., -5.1484, -0.8122, -1.8128],
        [-5.9497, -3.0785, -4.8027,  ..., -5.9518, -1.9349, -2.6955],
        [-7.4916, -3.1528, -6.1113,  ..., -8.8033, -4.7496, -3.7616],
        ...,
        [-4.9416, -0.0169, -7.1110,  ..., -7.5699, -7.1176, -6.5637],
        [-7.1770, -4.6520, -0.0641,  ..., -6.6470, -

Starting experiment (model=GPN_LOP, dataset=AmazonPhotos, ood_type=None, split=1, init=1, results=./saved_experiments/classification/GPN_LOP/21/results_1.json, trained=True, evaluated=False).
Completed experiment (model=GPN_LOP, dataset=AmazonPhotos, ood_type=None, split=1, init=1, results=./saved_experiments/classification/GPN_LOP/21/results_1.json, trained=True, evaluated=False).


INFO - train_and_eval - Result: [Prediction(soft=tensor([[2.2804e-03, 5.6618e-02, 1.7264e-03,  ..., 2.9993e-03, 5.0717e-01,
         2.0611e-01],
        [3.9701e-04, 3.3742e-02, 1.7931e-02,  ..., 4.2209e-03, 1.3675e-01,
         1.2849e-02],
        [6.8812e-03, 5.4093e-02, 6.7021e-03,  ..., 5.6292e-04, 1.6903e-02,
         1.1880e-02],
        ...,
        [1.8468e-03, 9.9000e-01, 5.6091e-04,  ..., 3.7372e-03, 5.4735e-04,
         2.0189e-04],
        [1.6320e-01, 2.9940e-02, 7.2458e-01,  ..., 8.0213e-03, 9.7015e-03,
         4.5375e-03],
        [3.3498e-03, 1.6509e-01, 7.4905e-03,  ..., 6.8993e-03, 2.2861e-01,
         9.3803e-02]]), log_soft=tensor([[-6.0834, -2.8714, -6.3617,  ..., -5.8094, -0.6789, -1.5794],
        [-7.8315, -3.3890, -4.0212,  ..., -5.4677, -1.9896, -4.3545],
        [-4.9790, -2.9170, -5.0053,  ..., -7.4824, -4.0803, -4.4329],
        ...,
        [-6.2943, -0.0101, -7.4860,  ..., -5.5894, -7.5104, -8.5078],
        [-1.8128, -3.5085, -0.3222,  ..., -4.8257, -

In [80]:
dir(gpn_lop_pred)

['__annotations__',
 '__attrs_attrs__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'act_vec',
 'alpha',
 'alpha_features',
 'aux_log_soft',
 'aux_soft',
 'chi',
 'clone',
 'collate',
 'energy',
 'energy_feaures',
 'evidence',
 'evidence_ft',
 'evidence_ft_per_class',
 'evidence_nn',
 'evidence_node',
 'evidence_per_class',
 'ft_weight',
 'hard',
 'hidden',
 'hidden_features',
 'latent',
 'latent_features',
 'latent_node',
 'log_ft',
 'log_ft_per_class',
 'log_nn',
 'log_nn_per_class',
 'log_node',
 'log_prior',
 'log_q',
 'log_soft',
 'logits',
 'logits_features',
 'mu_1',
 'mu_1p',
 'mu_2',
 'mu_2p',
 'nn_weight',
 'p_c',
 'p_u',
 'p_uc',
 'pre

In [81]:
import pandas as pd
import networkx as nx


def generate_graph(data, preds, name):
    edges = data.edge_index.T.numpy()

    y = data.y.numpy()

    nodes = {
        "true_y": y,
        "train_mask": data.train_mask.numpy().astype(int),
        "val_mask": data.val_mask.numpy().astype(int),
        "test_mask": data.test_mask.numpy().astype(int),
    }

    for model, pred in preds.items():
        y_pred = pred.hard.numpy()
        err = (y != y_pred).astype(int)
        pc_aleatoric = pred.prediction_confidence_aleatoric.numpy()
        pc_epistemic = pred.prediction_confidence_epistemic.numpy()
        sc_aleatoric = pred.sample_confidence_aleatoric.numpy()
        sc_aleatoric_entropy = pred.sample_confidence_aleatoric_entropy.numpy()
        sc_epistemic = pred.sample_confidence_epistemic.numpy()
        sc_epistemic_entropy = pred.sample_confidence_epistemic_entropy.numpy()
        nodes[f"{model}_y"] = y_pred
        nodes[f"{model}_err"] = err
        nodes[f"{model}_pc_aleatoric"] = pc_aleatoric
        nodes[f"{model}_pc_epistemic"] = pc_epistemic
        nodes[f"{model}_sc_aleatoric"] = sc_aleatoric
        nodes[f"{model}_sc_aleatoric_entropy"] = sc_aleatoric_entropy
        nodes[f"{model}_sc_epistemic"] = sc_epistemic
        nodes[f"{model}_sc_epistemic_entropy"] = sc_epistemic_entropy

    g = nx.Graph()

    g.add_nodes_from(range(len(y)))

    for i in range(len(y)):
        node = g.nodes[i]
        for k, v in nodes.items():
            node[k] = v[i]

    g.add_edges_from(edges)

    nx.write_graphml(g, f"{name}.graphml")

    return g

g = generate_graph(data, {
    "gpn_rw": gpn_rw_pred,
    "gpn_lop": gpn_lop_pred
}, f"graphs/{dataset}")